## 0. Dataset and DataLoader

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import pickle
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)

        self.data = self.data.iloc[::5, :].reset_index(drop=True)

        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()

        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()

            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()

            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        history_weather = self.data.iloc[idx:idx + 288, 4:12].values.astype(float)  # [288, 8]
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)   # [288]
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)  # [24, 8]
        future_wind_power = self.data.iloc[idx + 312, 2]  # float

        return (
            torch.tensor(history_weather, dtype=torch.float32),
            torch.tensor(wind_power_history, dtype=torch.float32),
            torch.tensor(future_weather, dtype=torch.float32),
            torch.tensor(future_wind_power, dtype=torch.float32)
        )
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }
    
name='CAISO_zone_1_.csv'
with open('weather_scaler.pkl', 'rb') as f:
    weather_scaler = pickle.load(f)
dataset = WindPowerDataset(name, save_scalers=True)
wind_power_scaler=dataset.wind_power_scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=32
test_split=0.2
test_size = int(len(dataset) * test_split)
train_size = len(dataset) - test_size
train_dataset = Subset(dataset, list(range(train_size)))
test_dataset = Subset(dataset, list(range(train_size, len(dataset))))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1. Preparation

In [ ]:
with open('rup_ensemble_003_caiso.pkl', 'rb') as f:
    rup_ensemble_003 = pickle.load(f)

with open('rup_ensemble_005_caiso.pkl', 'rb') as f:
    rup_ensemble_005 = pickle.load(f)

with open('rup_ensemble_010_caiso.pkl', 'rb') as f:
    rup_ensemble_010 = pickle.load(f)

with open('RUPW010_caiso.pkl', 'rb') as f:
    rupw_010 = pickle.load(f)

with open('RUPW005_caiso.pkl', 'rb') as f:
    rupw_005 = pickle.load(f)

with open('RUPW003_caiso.pkl', 'rb') as f:
    rupw_003 = pickle.load(f)

In [ ]:
uap_loaded_list = []

uap_loaded_list.append(rup_ensemble_003)
uap_loaded_list.append(rup_ensemble_005)
uap_loaded_list.append(rup_ensemble_010)
uap_loaded_list.append(rupw_010)
uap_loaded_list.append(rupw_005)
uap_loaded_list.append(rupw_003)

## ablation experiment

### MLP without physical constraints

In [ ]:
class AttackDetectionDataset(Dataset):
    def __init__(self, data_loader, uap_list, scaler=None, max_samples=None):
        self.normal_samples = []
        self.attack_samples = []
        self.scaler = scaler

        print("Building dataset: collecting normal and attacked samples...")
        
        with torch.no_grad():
            count = 0
            for _, _, weather_future, _ in data_loader:
                B, T, F = weather_future.shape  # B, 24, 8
                weather_np = weather_future.cpu().numpy()
                self.normal_samples.append(weather_np)
                for uap_tensor in uap_list:
                    uap = uap_tensor.cpu().numpy()  # (24, 8)
                    attacked_batch = np.clip(weather_np + uap, 0, 1)  
                    self.attack_samples.append(attacked_batch)
                count += B
                if max_samples and count >= max_samples:
                    break

        self.normal_samples = np.concatenate(self.normal_samples, axis=0)
        self.attack_samples = np.concatenate(self.attack_samples, axis=0)
        n_normal = len(self.normal_samples)
        n_attack = len(self.attack_samples)
        min_len = min(n_normal, n_attack)
        self.normal_samples = self.normal_samples[:min_len]
        self.attack_samples = self.attack_samples[:min_len]

        print(f"Final dataset: {min_len} normal + {min_len} attacked = {2 * min_len} samples")

    def __len__(self):
        return len(self.normal_samples) + len(self.attack_samples)

    def __getitem__(self, idx):
        if idx < len(self.normal_samples):
            x = self.normal_samples[idx]
            y = 0  # normal
        else:
            x = self.attack_samples[idx - len(self.normal_samples)]
            y = 1  # attacked
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=24*8, hidden_dim=64, num_classes=2, dropout=0.3):
        super(SimpleMLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # x: (B, 24, 8) -> reshape to (B, 192)
        x = x.view(x.size(0), -1)
        return self.mlp(x)

In [ ]:
def train_mlp_detector():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    dataset = AttackDetectionDataset(
        data_loader=train_loader,  
        uap_list=uap_loaded_list,
        max_samples=None
    )

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader_cls = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader_cls = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = SimpleMLP().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    num_epochs = 20
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        y_true_train, y_pred_train = [], []

        for x, y in train_loader_cls:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pred = logits.argmax(dim=1)
            y_true_train.extend(y.cpu().numpy())
            y_pred_train.extend(pred.cpu().numpy())

        train_acc = accuracy_score(y_true_train, y_pred_train)
        train_f1 = f1_score(y_true_train, y_pred_train)

        model.eval()
        y_true_val, y_pred_val, y_probs_val = [], [], []
        val_loss = 0.0

        with torch.no_grad():
            for x, y in val_loader_cls:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = criterion(logits, y)
                val_loss += loss.item()

                probs = torch.softmax(logits, dim=1)[:, 1] 
                pred = logits.argmax(dim=1)

                y_true_val.extend(y.cpu().numpy())
                y_pred_val.extend(pred.cpu().numpy())
                y_probs_val.extend(probs.cpu().numpy())

        val_acc = accuracy_score(y_true_val, y_pred_val)
        val_f1 = f1_score(y_true_val, y_pred_val)
        val_auc = roc_auc_score(y_true_val, y_probs_val)

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {train_loss/len(train_loader_cls):.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f} | "
              f"Val Loss: {val_loss/len(val_loader_cls):.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_mlp_detector_caiso.pth")

    print("\n Final Validation Results:")
    print(classification_report(y_true_val, y_pred_val, target_names=['Normal', 'Attacked']))
    print(f"AUC Score: {val_auc:.4f}")

    return model, val_auc, val_acc, val_f1

In [ ]:
trained_model, auc, acc, f1 = train_mlp_detector()

In [ ]:
def evaluate_mlp_detector_on_test(
    wind_power_model_new,  
    test_loader,
    uap_loaded_list,
    device,
    model_path="best_mlp_detector_caiso.pth"
):

    print(f"Using device: {device}")

    model = SimpleMLP().to(device)
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        print("Loaded trained MLP detector weights.")
    except FileNotFoundError:
        raise FileNotFoundError(f"Model weights not found: {model_path}") 
    model.eval()

    wind_power_model_new = wind_power_model_new.to(device)
    wind_power_model_new.eval() 

    y_true_all = []
    y_pred_all = []
    y_probs_all = []
    attack_types = [] 

    criterion = nn.CrossEntropyLoss()
    test_loss = 0.0

    print("Starting comprehensive evaluation on multiple attack types...")

    with torch.no_grad():
        for batch_idx, (history_weather, wind_power_history, future_weather, future_wind_power) in enumerate(test_loader):
            B = future_weather.size(0)

            history_weather = history_weather.to(device)
            wind_power_history = wind_power_history.to(device)
            future_weather = future_weather.to(device)
            future_wind_power = future_wind_power.to(device)

            x_normal = future_weather
            logits_normal = model(x_normal.view(B, -1))
            loss = criterion(logits_normal, torch.zeros(B, dtype=torch.long).to(device))
            test_loss += loss.item()

            probs_normal = torch.softmax(logits_normal, dim=1)[:, 1].detach().cpu().numpy()
            pred_normal = logits_normal.argmax(dim=1).detach().cpu().numpy()

            y_true_all.extend([0] * B)
            y_pred_all.extend(pred_normal)
            y_probs_all.extend(probs_normal)
            attack_types.extend(['normal'] * B)

            for uap_tensor in uap_loaded_list:
                uap = uap_tensor.to(device)  # (24, 8)
                adv_weather_uap = torch.clamp(future_weather + uap.unsqueeze(0), 0, 1)

                logits_uap = model(adv_weather_uap.view(B, -1))
                probs_uap = torch.softmax(logits_uap, dim=1)[:, 1].detach().cpu().numpy()
                pred_uap = logits_uap.argmax(dim=1).detach().cpu().numpy()

                y_true_all.extend([1] * B)
                y_pred_all.extend(pred_uap)
                y_probs_all.extend(probs_uap)
                attack_types.extend(['uap'] * B)

            with torch.enable_grad():
                wind_power_model_new.train()
                future_weather_adv = future_weather.clone().detach().requires_grad_(True)
                pred_power = wind_power_model_new(wind_power_history, future_weather_adv)
                loss_attack = torch.nn.MSELoss()(pred_power.squeeze(), future_wind_power)

                if future_weather_adv.grad is not None:
                    future_weather_adv.grad.zero_()
                loss_attack.backward()

                epsilon = 0.03
                grad_sign = future_weather_adv.grad.data.sign()
                adv_weather_fgsm = torch.clamp(future_weather + epsilon * grad_sign, 0, 1)
                wind_power_model_new.eval()

                logits_fgsm = model(adv_weather_fgsm.view(B, -1))
                probs_fgsm = torch.softmax(logits_fgsm, dim=1)[:, 1].detach().cpu().numpy()
                pred_fgsm = logits_fgsm.argmax(dim=1).detach().cpu().numpy()

                y_true_all.extend([1] * B)
                y_pred_all.extend(pred_fgsm)
                y_probs_all.extend(probs_fgsm)
                attack_types.extend(['fgsm_new_model'] * B)

    test_loss = test_loss / len(test_loader)
    test_acc = accuracy_score(y_true_all, y_pred_all)
    test_f1 = f1_score(y_true_all, y_pred_all)
    test_auc = roc_auc_score(y_true_all, y_probs_all)

    def calc_metrics_for_type(types_to_include):
        idx = [i for i, atk in enumerate(attack_types) if atk in types_to_include]
        if len(idx) == 0:
            return {'acc': 0, 'f1': 0, 'auc': float('nan')}
        
        y_true = [y_true_all[i] for i in idx]
        y_pred = [y_pred_all[i] for i in idx]
        y_prob = [y_probs_all[i] for i in idx]

        if len(np.unique(y_true)) < 2:
            auc_score = float('nan')
        else:
            auc_score = roc_auc_score(y_true, y_prob)

        return {
            'acc': accuracy_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred),
            'auc': auc_score
        }

    results_by_type = {
        'overall': {'acc': test_acc, 'f1': test_f1, 'auc': test_auc},
        'normal_vs_uap': calc_metrics_for_type(['normal', 'uap']),
        'normal_vs_fgsm_new_model': calc_metrics_for_type(['normal', 'fgsm_new_model']),
    }

    print("\n" + "="*60)
    print("COMPREHENSIVE EVALUATION RESULTS")
    print("="*60)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Overall Accuracy:  {test_acc:.4f}")
    print(f"Overall F1-Score:  {test_f1:.4f}")
    print(f"Overall AUC:       {test_auc:.4f}")

    print("\n" + "-"*50)
    print("PERFORMANCE BY ATTACK TYPE")
    print("-"*50)
    for atk_type, metrics in results_by_type.items():
        if atk_type == 'overall':
            continue
        print(f"{atk_type.upper():15} | Acc={metrics['acc']:6.4f} | F1={metrics['f1']:6.4f} | AUC={metrics['auc']:6.4f}")

    print("\n" + "-"*50)
    print("Classification Report (Overall)")
    print("-"*50)
    print(classification_report(y_true_all, y_pred_all, target_names=['Normal (0)', 'Attacked (1)']))

    return {
        'overall': {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'auc': test_auc,
            'y_true': y_true_all,
            'y_scores': y_probs_all
        },
        'by_attack_type': results_by_type
    }

In [ ]:
class TemporalConvNet_V2(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        super(TemporalConvNet_V2, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]

            padding = (kernel_size - 1) * dilation_size

            conv = nn.Conv1d(
                in_channels, out_channels, kernel_size,
                stride=1, padding=padding, dilation=dilation_size  
            )
            relu = nn.ReLU()
            drop = nn.Dropout(dropout)

            if in_channels != out_channels:
                res_conv = nn.Conv1d(in_channels, out_channels, 1)
            else:
                res_conv = None

            layers.append(nn.ModuleDict({
                'conv': conv,
                'relu': relu,
                'dropout': drop,
                'res_conv': res_conv
            }))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        # x: (batch_size, in_channels, seq_len)
        for layer in self.layers:
            residual = x  

            x = layer['conv'](x)  
            x = layer['relu'](x)
            x = layer['dropout'](x)
            x = x[:, :, :residual.size(2)]  

            if layer['res_conv'] is not None:
                residual = layer['res_conv'](residual)
            x = x + residual
            x = layer['relu'](x)  
        return x


class WindPowerPredictorTCN_V2(nn.Module):
    def __init__(self, d_model=64, dropout=0.2, fusion_mode='concat'):
        super(WindPowerPredictorTCN_V2, self).__init__()
        self.fusion_mode = fusion_mode  # 'concat' or 'add'
        self.tcn_wind = TemporalConvNet_V2(
            num_inputs=1,
            num_channels=[d_model, d_model*2, d_model, d_model],  
            kernel_size=3,
            dropout=dropout
        )

        self.tcn_weather = TemporalConvNet_V2(
            num_inputs=8,
            num_channels=[d_model, d_model*2, d_model, d_model],
            kernel_size=3,
            dropout=dropout
        )

        if fusion_mode == 'concat':
            fc_input_dim = d_model * 2
        elif fusion_mode == 'add':
            fc_input_dim = d_model
        else:
            raise ValueError("fusion_mode must be 'concat' or 'add'")

        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, 288)
        # weather_future: (batch_size, 24, 8)

        x_wind = wind_history.unsqueeze(1)  # (B, 1, 288)
        x_wind = self.tcn_wind(x_wind)     # (B, D, 288)
        x_wind = x_wind[:, :, -1]          

        x_weather = weather_future.transpose(1, 2)  # (B, 8, 24)
        x_weather = self.tcn_weather(x_weather)     # (B, D, 24)
        x_weather = x_weather[:, :, -1]             # -> (B, D)

        if self.fusion_mode == 'concat':
            combined = torch.cat((x_wind, x_weather), dim=1)  # (B, 2D)
        elif self.fusion_mode == 'add':
            combined = x_wind + x_weather  

        output = self.fc(combined)         # (B, 1)
        output = self.sigmoid(output)
        return output

In [ ]:
attack_model=WindPowerPredictorTCN_V2().to(device)
attack_model.load_state_dict(torch.load('wind_tcn_caiso_sigmoid_v2.pth', map_location=device))

results = evaluate_mlp_detector_on_test(
    wind_power_model_new=attack_model,
    test_loader=test_loader,
    uap_loaded_list=uap_loaded_list,
    device=device,
    model_path="best_mlp_detector_caiso.pth"
)

## Ablation experiments targeting various physical constraints

In [ ]:
def compute_constraints(weather_future, weather_scaler, constraint_types=['c1', 'c2', 'c3']):
    B, T, F = weather_future.shape

    x_flat = weather_future.view(-1, F)
    x_unscaled_np = weather_scaler.inverse_transform(x_flat.cpu().numpy())
    x_unscaled = torch.tensor(x_unscaled_np, dtype=torch.float32, device=weather_future.device).view(B, T, F)

    DNI = x_unscaled[:, :, 1]           # Direct Normal Irradiance
    SZA = x_unscaled[:, :, 4]           # Solar Zenith Angle
    DHI = x_unscaled[:, :, 0]           # Diffuse Horizontal Irradiance
    GHI = x_unscaled[:, :, 2]           # Global Horizontal Irradiance
    Dew_Point = x_unscaled[:, :, 3]
    Temperature = x_unscaled[:, :, 7]
    RH = x_unscaled[:, :, 6]            # Relative Humidity

    constraints = []

    for c in constraint_types:
        if c == 'c1':
            cos_sza = torch.cos(SZA * torch.pi / 180)
            c1 = torch.abs((DNI * cos_sza + DHI) - GHI)
            constraints.append(c1.unsqueeze(-1))  # [B, T, 1]
        elif c == 'c2':
            c2 = torch.relu(Dew_Point - Temperature)
            constraints.append(c2.unsqueeze(-1))
        elif c == 'c3':
            a, b = 17.625, 243.04
            exp_dew = (a * Dew_Point) / (b + Dew_Point.clamp(min=1e-6))
            exp_t = (a * Temperature) / (b + Temperature.clamp(min=1e-6))
            RH_calculated = 100.0 * torch.exp(exp_dew - exp_t)
            c3 = torch.abs(RH - RH_calculated)
            constraints.append(c3.unsqueeze(-1))
        else:
            raise ValueError(f"Unknown constraint: {c}")

    if len(constraints) == 0:
        raise ValueError("At least one constraint must be specified")
    
    constraints = torch.cat(constraints, dim=-1)  # [B, T, N]
    return constraints

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind_power_model_path = 'wind_caiso_lstm_sigmoid_version2.pth'
wind_power_model = load_model(wind_power_model_path, device)

In [ ]:
class ConstraintDetector(nn.Module):
    def __init__(self, num_constraints, hidden_dim=64):
        super(ConstraintDetector, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_constraints * 24, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.fc(x)  # x: [B, 24 * num_constraints]

In [ ]:
def train_detector(
    train_loader: DataLoader,
    val_loader: DataLoader,
    weather_scaler,
    constraint_types,
    device='cuda',
    num_epochs=20,
    lr=1e-4,
    save_path=None
):
   
    print(f"============================================================")
    print(f"STARTING TRAINING: {''.join(constraint_types).upper()}")
    print(f"============================================================")
    print(f"Training Detector for constraints: {constraint_types}")

    model = ConstraintDetector(num_constraints=len(constraint_types)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCEWithLogitsLoss()

    best_val_acc = 0.0
    best_threshold = 0.5
    patience = 0
    max_patience = 5  

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            history_weather, wind_power_history, future_weather, future_wind_power = batch
            future_weather = future_weather.to(device)
            B = future_weather.size(0)
            labels = torch.zeros(B, device=device)
            uap_perturb = torch.rand_like(future_weather) * 0.01
            adv_weather = torch.clamp(future_weather + uap_perturb, 0, 1)
            adv_labels = torch.ones(B, device=device)
            mixed_weather = torch.cat([future_weather, adv_weather], dim=0)  # [2B, ...]
            mixed_labels = torch.cat([labels, adv_labels], dim=0)            # [2B]

            constraints = compute_constraints(mixed_weather.cpu(), weather_scaler, constraint_types)
            constraints_flat = constraints.view(constraints.size(0), -1).to(device)  # [2B, 24*C]

            logits = model(constraints_flat).squeeze()  # [2B]
            loss = criterion(logits, mixed_labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_probs = []
        val_true = []

        with torch.no_grad():
            for batch in val_loader:
                history_weather, wind_power_history, future_weather, future_wind_power = batch
                B = future_weather.size(0)
                future_weather = future_weather.to(device)

                constraints_normal = compute_constraints(future_weather.cpu(), weather_scaler, constraint_types)
                constraints_flat_normal = constraints_normal.view(B, -1).to(device)
                logits_normal = model(constraints_flat_normal).squeeze()
                prob_normal = torch.sigmoid(logits_normal).detach().cpu().numpy()  
                val_probs.append(prob_normal)
                val_true.append(np.zeros(B))

                uap_perturb = torch.rand_like(future_weather) * 0.01
                adv_weather = torch.clamp(future_weather + uap_perturb, 0, 1)
                constraints_adv = compute_constraints(adv_weather.cpu(), weather_scaler, constraint_types)
                constraints_flat_adv = constraints_adv.view(B, -1).to(device)
                logits_adv = model(constraints_flat_adv).squeeze()
                prob_adv = torch.sigmoid(logits_adv).detach().cpu().numpy()
                val_probs.append(prob_adv)
                val_true.append(np.ones(B))

        val_probs = np.concatenate(val_probs)
        val_true = np.concatenate(val_true)

        print(f"\n Debug Info at Epoch {epoch}:")
        print(f"  Total samples: {len(val_true)} (Normal: {len(val_true) - val_true.sum()}, Adv: {val_true.sum()})")
        print(f"  Mean normal prob: {np.mean(val_probs[val_true == 0]):.4f} ± {np.std(val_probs[val_true == 0]):.4f}")
        print(f"  Mean adv prob:    {np.mean(val_probs[val_true == 1]):.4f} ± {np.std(val_probs[val_true == 1]):.4f}")
        print(f"  Prob range: [{val_probs.min():.4f}, {val_probs.max():.4f}]")

        best_acc = 0.0
        best_thresh = 0.5
        thresholds = np.arange(0.01, 1.0, 0.01)

        for thresh in thresholds:
            pred_labels = (val_probs >= thresh).astype(int)
            acc = (pred_labels == val_true).sum() / len(val_true)
            if acc > best_acc:
                best_acc = acc
                best_thresh = thresh

        val_acc = best_acc

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_threshold = best_thresh  
            if save_path:
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'best_threshold': best_threshold,
                    'val_accuracy': best_val_acc,
                    'constraint_types': constraint_types
                }, save_path)
                print(f"Model saved to {save_path} "
                      f"(Accuracy = {best_val_acc:.4f}, Best Thresh = {best_threshold:.3f})")
            patience = 0
        else:
            patience += 1

        if epoch % 3 == 0:
            print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}, Thresh: {best_thresh:.3f}")

        if patience >= max_patience:
            print(f" Early stopping at epoch {epoch}")
            break

    print(f" Training finished. Best Val Accuracy: {best_val_acc:.4f} @ Thresh={best_threshold:.3f}")
    return model, best_threshold

In [ ]:
def evaluate_detector(
    model,
    test_loader,
    uap_loaded_list,
    weather_scaler,
    wind_power_model,
    constraint_types,
    device='cuda',
    model_path=None  
):

    if model_path is not None:
        print(f"Loading model from {model_path}")
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        best_threshold = checkpoint.get('best_threshold', 0.5)  
        print(f"Loaded best_threshold = {best_threshold:.4f}")
    else:
        best_threshold = 0.5
        print(f"No model_path provided. Using default threshold = {best_threshold:.3f}")

    model.eval()
    wind_power_model.to(device)
    wind_power_model.eval()

    all_pred_proba = []
    all_true_labels = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            history_weather, wind_power_history, future_weather, future_wind_power = batch
            B = future_weather.size(0)

            history_weather = history_weather.to(device)
            wind_power_history = wind_power_history.to(device)
            future_weather = future_weather.to(device)
            future_wind_power = future_wind_power.to(device)

            constraints_normal = compute_constraints(future_weather.cpu(), weather_scaler, constraint_types)
            constraints_flat_normal = constraints_normal.view(B, -1).to(device)
            logits_normal = model(constraints_flat_normal).squeeze()
            prob_normal = torch.sigmoid(logits_normal).detach().cpu().numpy()
            all_pred_proba.append(prob_normal)
            all_true_labels.append(np.zeros(B))

            for uap in uap_loaded_list:
                adv_weather = torch.clamp(future_weather + uap.to(device), 0, 1)
                constraints_adv = compute_constraints(adv_weather.cpu(), weather_scaler, constraint_types)
                constraints_flat_adv = constraints_adv.view(B, -1).to(device)
                logits_adv = model(constraints_flat_adv).squeeze()
                prob_adv = torch.sigmoid(logits_adv).detach().cpu().numpy()
                all_pred_proba.append(prob_adv)
                all_true_labels.append(np.ones(B))

            with torch.enable_grad():
                wind_power_model.train()
                future_weather_for_adv = future_weather.clone().detach().requires_grad_(True)
                pred_power = wind_power_model(wind_power_history, future_weather_for_adv)
                loss = torch.nn.MSELoss()(pred_power.squeeze(), future_wind_power)

                if future_weather_for_adv.grad is not None:
                    future_weather_for_adv.grad.zero_()
                loss.backward()

                grad_sign = future_weather_for_adv.grad.data.sign()
                epsilon = 0.03
                adv_weather_grad = torch.clamp(future_weather + epsilon * grad_sign, 0, 1)
                wind_power_model.eval()

                constraints_grad = compute_constraints(adv_weather_grad.cpu(), weather_scaler, constraint_types)
                constraints_flat_grad = constraints_grad.view(B, -1).to(device)
                logits_grad = model(constraints_flat_grad).squeeze()
                prob_grad = torch.sigmoid(logits_grad).detach().cpu().numpy()
                all_pred_proba.append(prob_grad)
                all_true_labels.append(np.ones(B))

    y_pred_proba = np.concatenate(all_pred_proba)
    y_true = np.concatenate(all_true_labels)

    y_pred = (y_pred_proba >= best_threshold).astype(int)

    correct = (y_pred == y_true).sum()
    total = len(y_true)
    accuracy = correct / total

    TP = ((y_pred == 1) & (y_true == 1)).sum()
    FP = ((y_pred == 1) & (y_true == 0)).sum()
    FN = ((y_pred == 0) & (y_true == 1)).sum()
    TN = ((y_pred == 0) & (y_true == 0)).sum()

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    def compute_auc_manual(y_true, y_score):
        indices = np.argsort(y_score)[::-1]
        y_true_sorted = y_true[indices]
        n_pos = y_true.sum()
        n_neg = len(y_true) - n_pos
        if n_pos == 0 or n_neg == 0:
            return 0.5
        tp = fp = auc = 0
        for label in y_true_sorted:
            if label == 1:
                tp += 1
            else:
                auc += tp
                fp += 1
        return auc / (n_pos * n_neg)

    auc = compute_auc_manual(y_true, y_pred_proba)

    print(f"\n" + "="*60)
    print(f"EVALUATION: Constraints = {constraint_types}")
    print(f"{'Metric':<12} {'Score':<10}")
    print(f"{'-'*20}")
    print(f"{'Accuracy':<12} {accuracy:.4f}")
    print(f"{'Precision':<12} {precision:.4f}")
    print(f"{'Recall':<12} {recall:.4f}")
    print(f"{'F1-Score':<12} {f1:.4f}")
    print(f"{'-'*20}")
    print(f"Total samples: {total} (Normal: {int(TN+FP)}, Adv: {int(TP+FN)})")
    print(f"Predictions: Predicted 0: {int(TN+FN)}, Predicted 1: {int(TP+FP)}")
    print(f"Confusion: TP={TP}, FP={FP}, FN={FN}, TN={TN}")
    print(f"Prob stats: [min={y_pred_proba.min():.4f}, max={y_pred_proba.max():.4f}, "
          f"mean={y_pred_proba.mean():.4f}]")
    print("="*60)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'y_true': y_true,
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred,
        'confusion': {'TP': int(TP), 'FP': int(FP), 'FN': int(FN), 'TN': int(TN)}
    }

In [ ]:
combinations = [
    ['c1'],
    ['c2'],
    ['c3'],
    ['c1', 'c2'],
    ['c1', 'c3'],
    ['c2', 'c3']
]

results = {}

for combo in combinations:
    name = "_".join(combo)
    save_path = f"{name}_detector_caiso.pth"

    print(f"\n{'='*60}")
    print(f"STARTING TRAINING: {name.upper()}")
    print(f"{'='*60}")

    trained_model, best_thresh = train_detector(  
        train_loader=train_loader,
        val_loader=test_loader,
        weather_scaler=weather_scaler,
        constraint_types=combo,
        device=device,
        save_path=save_path,
        lr=1e-4,
    )

In [ ]:
class WindPowerPredictorV2(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=3, dim_feedforward=128, dropout=0.1, max_seq_len=300):
        super(WindPowerPredictorV2, self).__init__()
        self.d_model = d_model

        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)

        self.pos_encoder_wind = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))
        self.pos_encoder_weather = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))


        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc_combine = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, T1)        e.g., (32, 288)
        # weather_future: (batch_size, T2, 8)    e.g., (32, 24, 8)

        batch_size, seq_len_wind = wind_history.shape
        _, seq_len_weather, _ = weather_future.shape

        if seq_len_wind > self.pos_encoder_wind.size(0):
            raise ValueError(f"Wind sequence length {seq_len_wind} exceeds max supported length {self.pos_encoder_wind.size(0)}")
        if seq_len_weather > self.pos_encoder_weather.size(0):
            raise ValueError(f"Weather sequence length {seq_len_weather} exceeds max supported length {self.pos_encoder_weather.size(0)}")

        wind_emb = self.embedding_wind(wind_history.unsqueeze(-1))           # (B, T1, D)
        weather_emb = self.embedding_weather(weather_future)                 # (B, T2, D)

        wind_emb = wind_emb.permute(1, 0, 2)  # (T1, B, D)
        weather_emb = weather_emb.permute(1, 0, 2)  # (T2, B, D)

        wind_emb = wind_emb + self.pos_encoder_wind[:seq_len_wind]      # (T1, B, D)
        weather_emb = weather_emb + self.pos_encoder_weather[:seq_len_weather]  # (T2, B, D)

        wind_features = self.transformer_encoder(wind_emb)              # (T1, B, D)
        weather_features = self.transformer_encoder(weather_emb)        # (T2, B, D)

        wind_last = wind_features[-1, :, :]       
        weather_last = weather_features[-1, :, :]  

        combined = torch.cat((wind_last, weather_last), dim=1)  # (B, 2D)

        output = self.fc_combine(combined)  # (B, 1)
        output = self.sigmoid(output)
        return output
    
model_path = 'wind_caiso_transformer_sigmoid_version2.pth'
wind_power_model = WindPowerPredictorV2().to(device)
wind_power_model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
wind_power_model.eval()  
combinations = [['c1'], ['c2'], ['c3'], ['c1', 'c2'], ['c1', 'c3']]

results = {}
print("Starting evaluation for all constraint combinations...")

for combo in combinations:
    name = "_".join(combo)
    save_path = f"{name}_detector_caiso.pth"
    print(f"\n{'='*60}")
    print(f"EVALUATING: {name.upper()}")
    print(f"{'='*60}")
    print(f"Loading detector from: {save_path}")

    detector = ConstraintDetector(num_constraints=len(combo)).to(device)

    try:
        checkpoint = torch.load(save_path, map_location=device)

        detector.load_state_dict(checkpoint['model_state_dict'])
        detector.eval()

        best_threshold = checkpoint.get('best_threshold', 0.5)
        print(f"Detector loaded. Best threshold = {best_threshold:.4f}")
        
    except FileNotFoundError:
        print(f"Model file not found: {save_path}")
        results[name] = None
        continue
    except KeyError as e:
        print(f"Missing key in checkpoint {save_path}: {e}")
        print("Hint: Make sure you're using the updated training code that saves 'best_threshold'.")
        results[name] = None
        continue
    except Exception as e:
        print(f"Error loading {save_path}: {e}")
        results[name] = None
        continue

    metrics = evaluate_detector(
        model=detector, 
        test_loader=test_loader,
        uap_loaded_list=uap_loaded_list,
        weather_scaler=weather_scaler,
        wind_power_model=wind_power_model,      
        constraint_types=combo,
        device=device,
        model_path=save_path  
    )
    results[name] = metrics
    print(f"Metrics for {name}: Accuracy={metrics['accuracy']:.4f}, AUC={metrics['auc']:.4f}")

print("\n All evaluations completed.")

## If there is noise in the experimental data

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path = 'wind_caiso_lstm_sigmoid_version2.pth'
model = load_model(model_path, device)

In [ ]:
def compute_constraints(weather_future, weather_scaler):
    batch_size, seq_len, feature_dim = weather_future.shape
    weather_future_reshaped = weather_future.view(-1, feature_dim)
    weather_future_unscaled = weather_scaler.inverse_transform(weather_future_reshaped.detach().cpu().numpy())
    weather_future_unscaled = torch.tensor(weather_future_unscaled, dtype=weather_future.dtype, device=weather_future.device)
    weather_future_unscaled = weather_future_unscaled.view(batch_size, seq_len, feature_dim)
    DNI = weather_future_unscaled[:, :, 1]
    Solar_Zenith_Angle = weather_future_unscaled[:, :, 4]
    DHI = weather_future_unscaled[:, :, 0]
    GHI = weather_future_unscaled[:, :, 2]
    Dew_Point = weather_future_unscaled[:, :, 3]
    Temperature = weather_future_unscaled[:, :, 7]
    Relative_Humidity = weather_future_unscaled[:, :, 6]
    cos_sza = torch.cos(Solar_Zenith_Angle * np.pi / 180.0)
    constraint_1 = torch.abs((DNI * cos_sza + DHI) - GHI)  # GHI = DNI*cos + DHI
    constraint_2 = torch.relu(Dew_Point - Temperature)     # Dew point <= Temperature
    constraint_3 = torch.abs(
        100 * torch.exp(17.625 * Dew_Point / (243.04 + Dew_Point)) /
        torch.exp(17.625 * Temperature / (243.04 + Temperature)) - Relative_Humidity
    )

    return torch.stack([constraint_1, constraint_2, constraint_3], dim=-1)

class ConstraintClassifier(nn.Module):
    def __init__(self, input_dim=72):
        super(ConstraintClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x
    


In [ ]:
def test_constraint_classifier(test_loader, uap_loaded_list, classifier_model, wind_power_model, weather_scaler, device='cuda', sensor_noise_std=0.0):
    wind_power_model.train()  
    classifier_model.eval()   

    all_uap_labels = []
    all_uap_preds = []
    all_bp_labels = []
    all_bp_preds = []

    epsilons = [0.03, 0.05, 0.10]

    add_noise = sensor_noise_std > 0
    noise_desc = f" (with sensor noise σ={sensor_noise_std})" if add_noise else ""

    print(f"Testing attack detection{noise_desc}")

    for history_weather, wind_history, weather_future, future_wind_power in test_loader:
        B = weather_future.size(0)
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)

        if add_noise:
            noise = torch.randn_like(weather_future) * sensor_noise_std
            weather_future_noisy = torch.clamp(weather_future + noise, 0, 1)
        else:
            weather_future_noisy = weather_future

        normal_constraints = compute_constraints(weather_future_noisy.cpu(), weather_scaler).to(device)
        normal_constraints = normal_constraints.view(B, -1)

        all_attacked_constraints = []
        for uap_loaded in uap_loaded_list:
            attacked_weather = torch.clamp(weather_future_noisy + uap_loaded.to(device), 0, 1)
            attacked_constraints = compute_constraints(attacked_weather.cpu(), weather_scaler).to(device)
            attacked_constraints = attacked_constraints.view(B, -1)
            all_attacked_constraints.append(attacked_constraints)

        if all_attacked_constraints:
            attacked_constraints_cat = torch.cat(all_attacked_constraints, dim=0)
            constraints_uap = torch.cat([normal_constraints, attacked_constraints_cat], dim=0)
            labels_uap = torch.cat([
                torch.ones(B, device=device),  
                torch.zeros(attacked_constraints_cat.size(0), device=device)  
            ])

            with torch.no_grad():
                outputs = classifier_model(constraints_uap).squeeze()
                preds = (outputs > 0.5).float()

            all_uap_labels.append(labels_uap.cpu().numpy())
            all_uap_preds.append(preds.cpu().numpy())

        wind_power_model.train()
        for param in wind_power_model.parameters():
            param.requires_grad = False

        weather_input_for_grad = weather_future_noisy.clone().detach().requires_grad_(True)
        pred_power = wind_power_model(wind_history, weather_input_for_grad)

        if pred_power.dim() == 1:
            pred_power = pred_power.unsqueeze(1)
        if future_wind_power.dim() == 1:
            future_wind_power = future_wind_power.unsqueeze(1)

        loss_power = nn.MSELoss()(pred_power.squeeze(), future_wind_power.squeeze())

        if loss_power.grad_fn is None:
            raise RuntimeError("loss_power has no grad_fn! Check if gradients are disabled.")

        wind_power_model.zero_grad()
        loss_power.backward()

        grad_sign = weather_input_for_grad.grad.data.sign()

        all_attacked_constraints_bp = []
        for eps in epsilons:
            adv_weather = torch.clamp(weather_input_for_grad + eps * grad_sign, 0, 1)
            attacked_constraints = compute_constraints(adv_weather.cpu(), weather_scaler).to(device)
            attacked_constraints = attacked_constraints.view(B, -1)
            all_attacked_constraints_bp.append(attacked_constraints)

        if all_attacked_constraints_bp:
            attacked_constraints_bp_cat = torch.cat(all_attacked_constraints_bp, dim=0)

            constraints_bp = torch.cat([normal_constraints, attacked_constraints_bp_cat], dim=0)
            labels_bp = torch.cat([
                torch.ones(B, device=device),
                torch.zeros(attacked_constraints_bp_cat.size(0), device=device)
            ])

            with torch.no_grad():
                outputs = classifier_model(constraints_bp).squeeze()
                preds = (outputs > 0.5).float()

            all_bp_labels.append(labels_bp.cpu().numpy())
            all_bp_preds.append(preds.cpu().numpy())

    def compute_metrics(true_labels, preds):
        true_labels = np.concatenate(true_labels)
        preds = np.concatenate(preds)

        acc = (preds == true_labels).mean() * 100
        precision = precision_score(true_labels, preds, average='weighted', zero_division=0) * 100
        recall = recall_score(true_labels, preds, average='weighted', zero_division=0) * 100
        f1 = f1_score(true_labels, preds, average='weighted', zero_division=0) * 100

        return acc, precision, recall, f1

    if all_uap_labels:
        uap_acc, uap_prec, uap_rec, uap_f1 = compute_metrics(all_uap_labels, all_uap_preds)
        print(f'UAP Attack Detection Results{noise_desc}:')
        print(f'   Accuracy:  {uap_acc:.2f}%')
        print(f'   Precision: {uap_prec:.2f}%')
        print(f'   Recall:    {uap_rec:.2f}%')
        print(f'   F1-Score:  {uap_f1:.2f}%')
    else:
        uap_acc = uap_prec = uap_rec = uap_f1 = 0
        print('No UAP samples evaluated.')

    if all_bp_labels:
        bp_acc, bp_prec, bp_rec, bp_f1 = compute_metrics(all_bp_labels, all_bp_preds)
        print(f'FGSM (Backpropagation) Attack Detection Results{noise_desc}:')
        print(f'   Accuracy:  {bp_acc:.2f}%')
        print(f'   Precision: {bp_prec:.2f}%')
        print(f'   Recall:    {bp_rec:.2f}%')
        print(f'   F1-Score:  {bp_f1:.2f}%')
    else:
        bp_acc = bp_prec = bp_rec = bp_f1 = 0
        print('No BP (FGSM) samples evaluated.')

    return uap_acc, uap_prec, uap_rec, uap_f1, bp_acc, bp_prec, bp_rec, bp_f1

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
#wind_caiso_transformer_sigmoid.pth
model_path = 'wind_caiso_transformer_sigmoid.pth'
wind_power_model= load_model(model_path, device)

In [ ]:
classifier_model = ConstraintClassifier(input_dim=72)
classifier_model.load_state_dict(torch.load('CAISO_zone_1_PALM.pth', map_location=device))

In [ ]:
noise_stds = [0.001, 0.002, 0.003, 0.005]
results = []

for noise_std in noise_stds:
    print(f"\nRunning test with sensor_noise_std = {noise_std}")
    metrics = test_constraint_classifier(
        test_loader=test_loader,
        uap_loaded_list=uap_loaded_list,
        classifier_model=classifier_model,
        wind_power_model=wind_power_model,
        weather_scaler=weather_scaler,
        device='cuda',
        sensor_noise_std=noise_std
    )
    results.append((noise_std, metrics))